# SCAnalysis for single cell RNA-seq

This notebook details the usage of SCAnalysis for single cell RNA-seq data.

To view directly: https://nbviewer.jupyter.org/github/helenjin/scanalysis/blob/master/notebooks/SCAnalysis.ipynb
*(ignore if already here)*

## Table of Contents

1. [Introduction](#intro)
2. [Loading Data](#loading)
3. [Data Preprocessing](#preprocessing):
    *a. [Data Filtering](#filter)
    b. [Data Normalization](#norm)
    c. [PCA](#pca)
    d. [Diffusion maps](#dmap)
    e. [tSNE](#tsne)*
4. [Saving data](#savedata)
5. [General Plots](#genplot):
    *a. [PCA visualization](#pcavisual)
    b. [tSNE visualization](#tsnevisual)
    c. [Diffusion maps visualization](#dmvisual)*   
6. [Gene Set Enrichment Analysis (GSEA)](#gsea)
7. [Running Wishbone](#wishbone)
8. [Plotting Wishbone Results](#wbplot)
9. [Running MAGIC](#magic)
10. [Plotting MAGIC Results](#mgplot)
11. [Saving figures](#savefig)
12. [Running Palantir](#palantir)
13. [Plotting Palantir Results](#prplot)
14. [References](#ref)

<a id="intro"></a>
## Introduction

SCAnalysis is a package for analyzing single cell data. It includes the Wishbone, MAGIC, and Palantir packages:

* Wishbone is an algorithm to identify bifurcating developmental trajectories from single cell data. Wishbone can applied to single cell RNA-seq (as for mass cytometry datasets--> not currently)

* MAGIC (Markov-Affinity Based Graph Imputation of Cells) is an interactive tool to impute missing values in single-cell data and restore the structure of the data. It also provides data preprocessing functionality such as dimensionality reduction and gene expression visualization.

* Palantir -tbd

<a id="loading"></a>
## Loading Data

First, import the package.

In [ ]:
import scanalysis

Then, you can load the data using the load function in the loadsave file of the io folder. Here, we will be using the sample_scseq_data.csv data provided in the data folder as an example.

In [ ]:
df = scanalysis.io.loadsave.load("~/scanalysis/data/sample_scseq_data.csv")

Also, import plotting and miscellaneous.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

<a id="preprocessing"></a>
## Data preprocessing 

<a id="filter"></a>
### Data filtering

In [ ]:
fig, ax = scanalysis.plots.plot.plot_molecules_per_cell_and_gene(df)

From these histograms, choose the appropriate cutoffs to filter the data. In this case, the data has already been filtered.

In [ ]:
# Minimum molecules/cell value
CELL_MIN = 0

# Maximum molecules/cell values
CELL_MAX = 1000000

# Minimum number of nonzero cells/gene 
# (None if no filtering desired)
GENE_NONZERO = None

# Minimum number of molecules/gene
# (None if no filtering desired)
GENE_MOLECULES = None

In [ ]:
df = scanalysis.io.preprocess.filter_scseq_data(df, filter_cell_min=CELL_MIN, filter_cell_max=CELL_MAX, 
                         filter_gene_nonzero=GENE_NONZERO, filter_gene_mols=GENE_MOLECULES)

<a id="norm"></a>
### Data normalization

In [ ]:
data = scanalysis.io.preprocess.normalize_scseq_data(df)

<a id="pca"></a>
### Principal Component Analysis (PCA)

The first step in data processing for Wishbone is to determine metagenes using principal component analysis. This representation is necessary to overcome the extensive dropouts that are pervasive in single cell RNA-seq data.

For a visual representation of PCA results, see [PCA visualization](#pcavisual). *However, note that the PCA visualization functions already run PCA within themselves, so there is no need to run PCA separately beforehand.*

In [ ]:
r1, r2 = scanalysis.utils.pca.run_pca(data)

##### *Note: This sample dataset is especially sensitive, so we will be using the PCA of the original Wishbone package. (as shown below)

temp is the data after PCA is run on it.

In [ ]:
import wishbone
import os

scdata = wishbone.wb.SCData.from_csv(os.path.expanduser('~/.wishbone/data/sample_scseq_data.csv'), data_type='sc-seq', normalize=True)
scdata.run_pca()

In [ ]:
from copy import deepcopy
import numpy as np
import pandas as pd

n_pca_components = 5
temp = deepcopy(scdata.data)
temp -= np.min(np.ravel(temp))
temp /= np.max(np.ravel(temp))
temp = pd.DataFrame(np.dot(temp, scdata.pca['loadings'].iloc[:, 0:n_pca_components]),
                    index=scdata.data.index)

<a id="dmap"></a>
### Diffusion Maps

Diffusion maps is a non-linear dimensionality reduction technique to denoise the data and capture the major axes of variation. Diffusion maps can be determined by using the run_diffusion_map function and the diffusion components visualized on tSNE maps using plot_diffusion_components. See [Diffusion map visualization](#dmvisual)

Note: PCA must be run separately on data before diffusion maps (i.e. PCA is not included in diffusion maps function)

In [ ]:
tempEigvec, tempEigval = scanalysis.utils.diffusionmap.run_diffusion_map(temp)

<a id="tsne"></a>
### tSNE

Note: [PCA](#pca) must be run separately on data before tSNE (i.e. PCA is not included in tSNE function)

For a visual representation of tSNE results, see [tSNE visualization](#tsnevisual)

In [ ]:
t = scanalysis.utils.tsne.TSNE()
d = t.fit_transform(temp)

In [ ]:
t1 = scanalysis.utils.tsne.TSNE()
d1 = t1.fit_transform(r1)

<a id="savedata"></a>
## Saving Data **might need to revise

Data can be saved to a pickle file and loaded using the save and load functions. 

In [ ]:
scanalysis.io.loadsave.save(data, 'mouse_marrow_scdata.p')
p = scanalysis.io.loadsave.load('mouse_marrow_scdata.p')

<a id="genplot"></a>
## General Plots

<a id="pcavisual"></a>
### PCA visualization

*Note: Run the plot_pca_variance_explained function WITHOUT running PCA on the data beforehand, since PCA will be run automatically.*

Results shown below for plot_pca_variance_explained_v1, which is Wishbone's version of the function.

In [ ]:
fig, ax = scanalysis.plots.plot.plot_pca_variance_explained_v1(data, n_components=40, random=True)

Results shown below for plot_pca_variance_explained_v2, which is MAGIC's version of the function.

In [ ]:
fig, ax = scanalysis.plots.plot.plot_pca_variance_explained_v2(data, n_components=40, random=True)

<a id="tsnevisual"></a>
### tSNE visualization

Wishbone uses [tSNE](#tsne) for visualization and tSNE can be run using the run_tsne function which takes the number of principal components as the parameter. From the above plot, 5 seems an appropriate number of components to use.

tSNE results can be visualized by the plot_tsne and plot_tsne_by_cell_sizes functions. The plot_tsne_by_cell_sizes function colors the cells by their molecule counts before normalization.

In [ ]:
fig, ax = scanalysis.plots.plot.plot_tsne(d1)

In [ ]:
fig = plt.figure(figsize=[5, 4])
scanalysis.plots.plot.plot_tsne_by_cell_sizes(df, d1, fig = fig)

In [ ]:
fig, ax = scanalysis.plots.plot.plot_gene_expression(data, d, genes = ['CD34', 'GATA2', 'GATA1', 'MPO'])

<a id="dmvisual"></a>
### Diffusion map visualization

*Note: Please run [diffusion maps](#dmap) and [tSNE](#tsne) before plotting diffusion components (via plot_diffusion_components function).*

In [ ]:
fig, ax = scanalysis.plots.plot.plot_diffusion_components(d, tempEigvec, tempEigval)

The run_diffusion_map_correlations function is designed to work for single cell RNA-seq (not mass-cyt).
Please run diffusion maps using run_diffusion_map before determining correlations.

Note: the component 0 is the trivial component and does not encode any information of the data.

In [ ]:
dmap_corr = scanalysis.plots.plot.run_diffusion_map_correlations(data, tempEigvec)

After determining the diffusion map correlations, we can plot the gene component correlations (via plot_gene_component_correlations function).

In [ ]:
scanalysis.plots.plot.plot_gene_component_correlations(dmap_corr)

<a id="gsea"></a>
## Gene Set Enrichment Analysis (GSEA)
For more info on the original software, see [GSEA](http://software.broadinstitute.org/gsea/index.jsp)

The enrichments can be determined using the run_gsea function. This function needs the prefix for generating GSEA reports and a gmt file representing the different gene sets. The following invocation of the function shows the supported set of gmt files.

*Note: Please make sure to run run_diffusion_map_correlations() before running GSEA to annotate those components.*

Note: The gmt files package with Wishbone/SCAnalysis assume all the gene names to be upper case. This can be ensured using the following code to convert them to upper case.

In [ ]:
data.columns = data.columns.str.upper()

In [ ]:
scanalysis.tools.wb.gsea.run_gsea(dmap_corr, output_stem= os.path.expanduser('~/.scanalysis/tools/gsea/mouse_marrow'))

Since this is data from mouse, gmt_file parameter can be set to (mouse, gofat.bp.v1.0.gmt.txt)

In [ ]:
reports = scanalysis.tools.wb.gsea.run_gsea(dmap_corr, output_stem= os.path.expanduser('~/.scanalysis/gsea/mouse_marrow'), 
                          gmt_file=('mouse', 'gofat.bp.v1.0.gmt.txt'))

The detailed reports can be found at ~/.wishbone/gsea/

In [ ]:
!open ~/.scanalysis/gsea/

run_gsea function also returns the top enrichment gene sets along each component. GSEA determines enrichments that are either positively or negatively correlated with the gene component correlations. In this dataset, components 1 and 2 show relevant enrichments and are used for running Wishbone/SCAnalysis. Please see Selection of diffusion components for single cell RNA-seq section of the Supplementary Methods for more details.

In [ ]:
# Component 1 enrichments
reports[1]['neg']

In [ ]:
# Component 2 enrichments
reports[2]['pos']

<a id="wishbone"></a>
## Running Wishbone

For a visual representation of results, see [Plotting Wishbone Results](#wbplot)

First, create an instance of the Wishbone class.

In [ ]:
w = scanalysis.tools.wb.wishbone.Wishbone()

After initialization, Wishbone can be run by specifying the start cell and number of waypoints to be used. The start cell for this dataset was chosen based on high expression of CD34. (for each dataset, there is a corresponding start cell particular to that dataset)

*Note: Keep in mind that Wishbone requires data that has been run through [normalization](#norm), [PCA](#pca), and [diffusion maps](#dmap).*

Here, we will consider only 2 components.(?)

In [ ]:
w.wishbone(tempEigvec.iloc[:,[1,2]], 'W30258', k=15, l=15, num_waypoints =250, branch=True)

<a id="wbplot"></a>
## Plotting Wishbone Results

Wishbone trajectory and branch results can be visualized on tSNE maps using the plot_wishbone_on_tsne function.

*Note: Please make sure to run [Wishbone](#wishbone) before attempting to plot Wishbone results.*

In [ ]:
scanalysis.plots.wb_plot.plot_wishbone_on_tsne(w, d)

Gene expression trends along the Wishbone trajectory can be visualized using the plot_marker_trajectory function. This function also returns the smoothed trends along with the matplotlib fig, ax handler objects.

Note: Variance calculation is currently not supported for single-cell RNA-seq (sc-seq)

In [ ]:
vals, fig, ax = scanalysis.plots.wb_plot.plot_marker_trajectory(data, w, ['CD34', 'GATA1', 'GATA2', 'MPO']);

The marker trends can be visualized as heatmaps in a given trajectory range using the following functions:

In [ ]:
scanalysis.plots.wb_plot.plot_marker_heatmap(w, vals)

In [ ]:
scanalysis.plots.wb_plot.plot_marker_heatmap(w, vals, trajectory_range=[0.1, 0.6])

The change in marker trends along the trajectory or derivatives can be visualized using these functions:

In [ ]:
scanalysis.plots.wb_plot.plot_derivatives(w, vals)

In [ ]:
scanalysis.plots.wb_plot.plot_derivatives(w, vals, trajectory_range=[0.3, 0.6])

<a id="magic"></a>
## Running MAGIC

For a visual representation of MAGIC results, see [Plotting MAGIC Results](#mgplot)

MAGIC can be run with the run_magic function.

*Note: Data should be [filtered](#filter) and [normalized](#norm) before running MAGIC. Running PCA is not necessary, since the run_magic function automatically performs PCA.*

In [ ]:
new_data = scanalysis.tools.magic.run_magic(data)

#### Let's try MAGIC with the data set used in the original MAGIC [notebook](http://nbviewer.jupyter.org/github/pkathail/magic/blob/develop/notebooks/Magic_single_cell_RNAseq.ipynb).

In [ ]:
import scanalysis

In [ ]:
m_data = scanalysis.io.loadsave.load("~/sdata_nn_TGFb_day_8_10.csv")

We have to (filter and) normalize the data.

In [ ]:
# Minimum molecules/cell value
CELL_MIN = 0

# Maximum molecules/cell values
CELL_MAX = 1000000

# Minimum number of nonzero cells/gene 
# (None if no filtering desired)
GENE_NONZERO = None

# Minimum number of molecules/gene
# (None if no filtering desired)
GENE_MOLECULES = None

m_data = scanalysis.io.preprocess.filter_scseq_data(m_data, filter_cell_min=CELL_MIN, filter_cell_max=CELL_MAX, 
                         filter_gene_nonzero=GENE_NONZERO, filter_gene_mols=GENE_MOLECULES)
## ^but this takes forever...
### also Pooja doesn't actually filter the data in the example notebook 
## (there are just dummy parameters as an example of how to call the filtering function), so you should still be able to tes

In [ ]:
m_data = scanalysis.io.preprocess.normalize_scseq_data(m_data)

Then, let's apply the run_magic function on m_data.

In [ ]:
new_m_data = scanalysis.tools.magic.run_magic(m_data)

<a id="mgplot"></a>
## Plotting MAGIC Results

*Note: Please make sure to run [MAGIC](#magic) on normalized data before attempting to plot various MAGIC results.*

### Gene-gene scatter plots

#### 2D scatter plot before MAGIC:

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(data, ['SRRM1', 'TAB2'], color = 'GPX4')
ax.set_xlabel('SRRM1')
ax.set_ylabel('TAB2')

The second plot below is for the data set used in the original MAGIC [notebook](http://nbviewer.jupyter.org/github/pkathail/magic/blob/develop/notebooks/Magic_single_cell_RNAseq.ipynb).

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(m_data, ['VIM', 'CDH1'], color='ZEB1')
ax.set_xlabel('Vimentin (VIM)')
ax.set_ylabel('E-cadherin (CDH1)')

#### 2D scatter plot after MAGIC:

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(new_data, ['MAGIC SRRM1', 'MAGIC TAB2'], color = 'MAGIC GPX4')
ax.set_xlabel('MAGIC SRRM1')
ax.set_ylabel('MAGIC TAB2')

The second plot below is for the data set used in the original MAGIC [notebook](http://nbviewer.jupyter.org/github/pkathail/magic/blob/develop/notebooks/Magic_single_cell_RNAseq.ipynb).

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(new_m_data, ['MAGIC VIM', 'MAGIC CDH1'], color ='MAGIC ZEB1')
ax.set_xlabel('MAGIC Vimentin (VIM)')
ax.set_ylabel('MAGIC E-cadherin (CDH1)')

#### 3D scatter plot before MAGIC:

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(data, ['SRRM1', 'TAB2', 'CBR1'], color='GPX4')
ax.set_xlabel('SRRM1')
ax.set_ylabel('TAB2')
ax.set_zlabel('CBR1')

The second plot below is for the data set used in the original MAGIC [notebook](http://nbviewer.jupyter.org/github/pkathail/magic/blob/develop/notebooks/Magic_single_cell_RNAseq.ipynb).

***does this look weird?**

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(m_data, ['VIM', 'CDH1', 'FN1'], color='ZEB1')
ax.set_xlabel('Vimentin (VIM)')
ax.set_ylabel('E-cadherin (CDH1)')
ax.set_zlabel('Fibronectin (FN1)')

#### 3D scatter plot after MAGIC:

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(new_data, ['MAGIC SRRM1', 'MAGIC TAB2', 'MAGIC CBR1'], color='MAGIC GPX4')
ax.set_xlabel('MAGIC SRRM1')
ax.set_ylabel('MAGIC TAB2')
ax.set_zlabel('MAGIC CBR1')

The second plot below is for the data set used in the original MAGIC [notebook](http://nbviewer.jupyter.org/github/pkathail/magic/blob/develop/notebooks/Magic_single_cell_RNAseq.ipynb).

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(new_m_data, ['MAGIC VIM', 'MAGIC CDH1', 'MAGIC FN1'], color='MAGIC ZEB1')
ax.set_xlabel('MAGIC Vimentin (VIM)')
ax.set_ylabel('MAGIC E-cadherin (CDH1)')
ax.set_zlabel('MAGIC Fibronectin (FN1)')
ax.set_zlim(35, 150)

### PCA scatter plots

#### PC2 vs PC3 colored by CDH1, VIM, FN1 and ZEB1 (before MAGIC):

In [ ]:
scanalysis.plots.plot.FigureGrid(2).savefig('h')

In [ ]:
gs = gridspec.GridSpec(2,2)
fig = plt.figure(figsize=[15, 12])
genes = ['SRRM1', 'TAB2', 'CBR1', 'GPX4']
for i in range(len(genes)):
    ax = fig.add_subplot(gs[i//2, i%2])
    scanalysis.plots.plot.scatter_gene_expression(data, genes=['PC2', 'PC3'], color=genes[i], fig=fig, ax=ax)

#### PC2 vs PC3 colored by CDH1, VIM, FN1 and ZEB1 (after MAGIC):

### tSNE scatter plots

#### tSNE maps colored by CDH1, VIM, FN1, and ZEB1 (before MAGIC):

#### tSNE maps colored by CDH1, VIM, FN1, and ZEB1 (after MAGIC):

<a id="savefig"></a>
## Saving figures
You can save a figure as a png file using "savefig" as shown below.

In [ ]:
scanalysis.plots.plot.savefig(fig, 'h')

### original MAGIC package....

In [ ]:
fig, ax = scanalysis.plots.plot.scatter_gene_expression(scdata.data, ['SRRM1', 'TAB2'], color = 'GPX4')

In [ ]:
fig, ax = scdata.scatter_gene_expression(['SRRM1', 'TAB2'], color = 'GPX4')
ax.set_xlabel('SRRM1')
ax.set_ylabel('TAB2')

In [ ]:
scdata.run_magic()

In [ ]:
scdata.magic.scatter_gene_expression(['SRRM1', 'TAB2'], color = 'GPX4')

In [ ]:
fig, ax = scdata.scatter_gene_expression(['SRRM1', 'TAB2', 'CBR1'], color='GPX4')
ax.set_xlabel('SRRM1')
ax.set_ylabel('TAB2')
ax.set_zlabel('CBR1')

In [ ]:
fig, ax = scdata.magic.scatter_gene_expression(['SRRM1', 'TAB2', 'CBR1'], color='GPX4')
ax.set_xlabel('SRRM1')
ax.set_ylabel('TAB2')
ax.set_zlabel('CBR1')

<a id="palantir"></a>
## Running Palantir

First, load the pickle file with the data. The data is normalized and log transformed. 

Use the following parameters for this particular data set:
* start_cell: Run5_126835192163230
* num_waypoints: 300
* flock = 0

In [1]:
import scanalysis

In [2]:
mb_data = scanalysis.io.loadsave.load("~/mb_data.p")

Successfully loaded /Users/hjin/mb_data.p


In [4]:
import pandas as pd

In [5]:
mb = pd.DataFrame.transpose(mb_data)

mb

,KCTD15,STT3B,NAT6,FHL2,SP140L,DOCK9,CENPN,NEDD9,HIST1H2AM,TEK,...,ZNF543,SGTB,EEF1A1,TSPAN33,DNAJC4,SAG,FARP1,EXTL2,ZDHHC17,GOLPH3
Run4_120703408880541,-3.321928,-3.321928,-3.321928,0.171306,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.448183,-3.321928,1.105781,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703409056541,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.536806,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703409580963,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,5.660550,-3.321928,1.840569,-3.321928,-3.321928,-3.321928,0.880299,0.880299
Run4_120703423990708,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.120207,-3.321928,0.574557,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703424252854,-3.321928,-0.365265,-3.321928,-3.321928,-3.321928,-3.321928,-0.365265,-3.321928,-0.365265,-3.321928,...,-3.321928,-3.321928,6.740922,-3.321928,-3.321928,-3.321928,-3.321928,0.538690,-3.321928,-0.365265
Run4_120703436876077,-3.321928,-3.321928,0.363786,0.363786,0.363786,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.833304,-3.321928,1.306611,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703455025387,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-0.270688,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.987921,-0.270688,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120726911638237,-3.321928,0.918170,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-0.521655,-3.321928,...,-3.321928,-3.321928,6.682439,-3.321928,-0.521655,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120726912355038,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,...,-3.321928,-3.321928,6.426178,-3.321928,-0.098613,-3.321928,-3.321928,-0.098613,-3.321928,-3.321928
Run4_120726924974443,-3.321928,-0.910988,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928,-0.910988,-3.321928,-3.321928,...,-3.321928,-3.321928,6.795428,-3.321928,-3.321928,-3.321928,-3.321928,-0.910988,-0.910988,-0.910988


In [6]:
DMEigVals = scanalysis.io.loadsave.load("~/palantir/dm_eig_vals.csv")

Successfully loaded /Users/hjin/palantir/dm_eig_vals.csv


In [7]:
DMEigs = scanalysis.io.loadsave.load("~/palantir/dm_eigs.csv")

Successfully loaded /Users/hjin/palantir/dm_eigs.csv


Note: the run_multibranch function takes ~10 minutes.

In [ ]:
atrajectory = scanalysis.io.loadsave.load("~/palantir/trajectory.csv")

In [ ]:
#pca
r1, r2 = scanalysis.utils.pca.run_pca(mb_data) 

#tsne
t1 = scanalysis.utils.tsne.TSNE()
d1 = t1.fit_transform(r1)

#plot tsne
fig, ax = scanalysis.plots.plot.plot_tsne(d1)

^ not exactly sure if this is correct

In [8]:
res = scanalysis.tools.pr.palantir.run_multibranch(data_ = mb, DMEigs = DMEigs, DMEigVals = DMEigVals, dm_eigs = [1,2], start_cell="Run5_126835192163230", num_waypoints = 300, flock = 0)


Sampling and flocking waypoints...
Time for determining waypoints: 0.002096748352050781 minutes
Shortest path distances...
Time for shortest paths: 11.152179503440857 minutes
Determining perspectives, trajectory...
Correlation at iteration 1: 1.0000
Determining terminal states...
Entropy and branch probabilities...
Markov chain construction...
Computing fundamental matrix and absorption probabilities...
Project results to all cells...


/usr/local/lib/python3.6/site-packages/scanalysis/tools/pr/palantir.py:162: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = hierarchy.linkage(pairwise_distances(data.loc[cells,:]))
/usr/local/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [10]:
res['trajectory']

Run4_120703408880541    2.286934
Run4_120703409056541    0.074342
Run4_120703409580963    0.700627
Run4_120703423990708    0.694064
Run4_120703424252854    0.858022
Run4_120703436876077    0.303698
Run4_120703455025387    0.677564
Run4_120726911638237    0.739607
Run4_120726912355038    0.537471
Run4_120726924974443    0.639494
Run4_120726924978030    0.482449
Run4_120726943295348    0.709921
Run4_120726943845302    0.736320
Run4_120772961130853    0.345368
Run4_120786758780660    0.321040
Run4_120786786086116    0.204759
Run4_120786804205803    0.743148
Run4_120786804401973    0.572899
Run4_120797898099934    0.772021
Run4_120797925428019    0.513296
Run4_120797944278244    2.411774
Run4_120797944309486    2.224210
Run4_120797944462237    0.439472
Run4_120797944538971    1.667476
Run4_120797945084765    0.302636
Run4_120864497952619    0.966163
Run4_121202296155437    2.509334
Run4_121202296712412    0.880035
Run4_121202296875939    0.110837
Run4_121202311609131    0.682364
          

Compare trajectory results with the trajectory.csv file using a scatter plot, shown below.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(res['trajectory'], atrajectory)

<a id="prplot"></a>
## Plotting Palantir Results

In [6]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

First, create an instance of the DiffEntrResults class.

In [12]:
der = scanalysis.plots.pr_plot.DiffEntrResults(trajectory = res['trajectory'], branches = None, branch_prob=res['branch_probs'], no_bins=500)

Here, we will try the first five genes in the data matrix. You shouldn't plot all the markers because that will kill your computer.  

In [8]:
mb_data_f5 = mb_data.iloc[0:5,:]

We need the data to be in cells x genes format (index x columns), so we will switch the rows and columns.

In [34]:
a = pd.DataFrame.transpose(mb_data_f5)

In [35]:
a

,KCTD15,STT3B,NAT6,FHL2,SP140L
Run4_120703408880541,-3.321928,-3.321928,-3.321928,0.171306,-3.321928
Run4_120703409056541,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703409580963,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703423990708,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120703424252854,-3.321928,-0.365265,-3.321928,-3.321928,-3.321928
Run4_120703436876077,-3.321928,-3.321928,0.363786,0.363786,0.363786
Run4_120703455025387,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120726911638237,-3.321928,0.918170,-3.321928,-3.321928,-3.321928
Run4_120726912355038,-3.321928,-3.321928,-3.321928,-3.321928,-3.321928
Run4_120726924974443,-3.321928,-0.910988,-3.321928,-3.321928,-3.321928


Run the plot_markers function to visualize results.

In [13]:
der.branch_prob

,Run4_235626713532342,Run5_239477254471070
Run4_120703408880541,0.999998,0.000000
Run4_120703409056541,0.182656,0.817344
Run4_120703409580963,0.010092,0.989908
Run4_120703423990708,0.010411,0.989589
Run4_120703424252854,0.000000,0.998499
Run4_120703436876077,0.088207,0.911793
Run4_120703455025387,0.016095,0.983905
Run4_120726911638237,0.000000,0.992499
Run4_120726912355038,0.039662,0.960338
Run4_120726924974443,0.017445,0.982555


In [33]:
der.plot_markers(a)

Run4_235626713532342


KeyError: "None of [Index(['KCTD15', 'STT3B', 'NAT6', 'FHL2', 'SP140L'], dtype='object')] are in the [index]"

In [ ]:
mb_data_f5

In [ ]:
marker_trends[branch].loc[:, :] = np.ravel(trends).reshape([marker_data.shape[0], 
					len(self.traj_bins)])

In [13]:
mb_data_f5.shape[0]

5

In [14]:
import pandas as pd

In [25]:
der.branch_prob.columns

Index(['Run4_235626713532342', 'Run5_239477254471070'], dtype='object')

In [30]:
for branch in der.branch_prob:
    print(branch)

Run4_235626713532342
Run5_239477254471070


In [28]:
weights = der.branch_prob.loc[mb_data_f5.columns,'Run4_235626713532342' ]

In [29]:
weights

Run4_120703408880541    0.999997
Run4_120703409056541    0.144511
Run4_120703409580963    0.016099
Run4_120703423990708    0.016543
Run4_120703424252854    0.000000
Run4_120703436876077    0.078912
Run4_120703455025387    0.030889
Run4_120726911638237    0.021286
Run4_120726912355038    0.036232
Run4_120726924974443    0.023710
Run4_120726924978030    0.050166
Run4_120726943295348    0.015151
Run4_120726943845302    0.012988
Run4_120772961130853    0.070063
Run4_120786758780660    0.074233
Run4_120786786086116    0.119245
Run4_120786804205803    0.016530
Run4_120786804401973    0.033288
Run4_120797898099934    0.000000
Run4_120797925428019    0.053342
Run4_120797944278244    0.999999
Run4_120797944309486    0.999993
Run4_120797944462237    0.058725
Run4_120797944538971    0.999247
Run4_120797945084765    0.079301
Run4_120864497952619    0.000000
Run4_121202296155437    1.000000
Run4_121202296712412    0.000000
Run4_121202296875939    0.145640
Run4_121202311609131    0.017054
          

<a id="ref"></a>
## References

Setty M, Tadmor MD, Reich-Zeliger S, Angel O, Salame TM, Kathail P, Choi K, Bendall S, Friedman N, Pe’er D. "Wishbone identifies bifurcating developmental trajectories from single-cell data." Nat. Biotech. 2016 April 12. <http://dx.doi.org/10.1038/nbt.3569>

van Dijk, David, et al. "MAGIC: A diffusion-based imputation method reveals gene-gene interactions in single-cell RNA-sequencing data." BioRxiv (2017): 111591. <http://www.biorxiv.org/content/early/2017/02/25/111591>